In [1]:
#!/usr/bin/env python3
"""
DHN Performance Toolkit - Main Analysis Script
==============================================

District Heating Network performance analysis with:
- COP & Energy Analysis  
- Velocity Analysis
- Automated Reporting

Usage:
    1. Copy config_template.py to config_local.py
    2. Adjust paths in config_local.py
    3. Run: python main_analysis.py
"""

# Try to import local config, fallback to template
try:
    from config_local import SCENARIOS, DEFAULT_PARAMS
    print("✅ Using local configuration")
except ImportError:
    print("⚠️ config_local.py not found - using template")
    print("💡 Copy config_template.py to config_local.py and adjust paths")
    from config_template import SCENARIOS, DEFAULT_PARAMS



✅ Using local configuration


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# UESGraphs
from uesgraphs.uesgraph import UESGraph
import uesgraphs.analyze as analyze

In [3]:
from modules.decentral_pump_power import calculate_decentral_pump_energy

In [4]:
# Select scenario
scenario_key = "Scenario 1"  # Adjust this
scenario = SCENARIOS[scenario_key]

print(f"📋 Analyzing: {scenario['name']}")
print(f"📂 Output: {scenario['output_dir']}")

📋 Analyzing: Analysis 1
📂 Output: E:\rka_lko\work\2025_06_Hassel_simulation_analysis\Prio1_npro_complete_sfh\outputs


In [5]:
# Data Loading Functions
def get_dataframe(mask, file_path, uesgraph):
    """Load data for a specific mask pattern"""
    filter_list = []
    for node in uesgraph.nodelist_building:
        if not uesgraph.nodes[node]["is_supply_heating"]:
            name_bldg = uesgraph.nodes[node]["name"]
            filter_pattern = mask.format(name_bldg=name_bldg)
            filter_list.append(filter_pattern)
    
    df = analyze.process_simulation_result(file_path=file_path, filter_list=filter_list)
    df = analyze.prepare_DataFrame(
        df, 
        base_date=datetime.strptime(DEFAULT_PARAMS["start_date"], "%Y-%m-%d"), 
        end_date=datetime.strptime(DEFAULT_PARAMS["end_date"], "%Y-%m-%d"),
        time_interval=DEFAULT_PARAMS["time_interval"]
    )
    
    # Simplify column names
    import re
    pattern = re.compile(r'T([^.]+)')
    new_columns = []
    for col in df.columns:
        match = pattern.search(col)
        if match:
            new_columns.append(f"T{match.group(1)}")
        else:
            new_columns.append(col)
    df.columns = new_columns
    
    return df

# Load UESGraph
uesgraph = UESGraph()
uesgraph.from_json(path=scenario["json_path"], network_type="heating")
print(f"✅ Network loaded: {len(uesgraph.nodes)} nodes, {len(uesgraph.edges)} edges")

read nodes...
******
 input_ids were {'buildings': None, 'nodes': '5f4c976f-bea3-476b-875b-c4d08dfc2057', 'pipes': None, 'supplies': None}
...finished
✅ Network loaded: 68 nodes, 67 edges


In [ ]:
# Load pump power data
PUMP_MASK = 'networkModel.demandT{name_bldg}.simplePumpConstFlow.pumpHeating.P$'
df_pump_power = get_dataframe(PUMP_MASK, scenario["data_path"], uesgraph)

print(f"📊 Pump data loaded: {df_pump_power.shape}")
print(f"📊 Buildings: {list(df_pump_power.columns)}")
# Run pump analysis using the module
pump_results = calculate_decentral_pump_energy(
    df_pump_power, 
    output_dir=scenario["output_dir"],
    store_result=True,
)


Processing: E:\rka_lko\work\2025_06_Hassel_simulation_analysis\Prio1_npro_complete_sfh\Sim20250612_154050_1\Results\Sim20250612_154050_1_inputs.gzip
📊 Pump data loaded: (35041, 32)
📊 Buildings: ['T56_wa2b_efh2', 'T54_wa2b_efh1', 'T48_wa2b_rhh3_2', 'T50_wa2b_rhh3_3', 'T44_wa2b_dhh6_1', 'T42_wa2b_dhh6_2', 'T58_wa2b_dhh3_1', 'T36_wa2b_dhh3_2', 'T38_wa2b_dhh4_1', 'T40_wa2b_dhh4_2', 'T42_wa2b_dhh2_1', 'T34_wa2b_dhh2_2', 'T32_wa2b_rhh2_1', 'T30_wa2b_rhh2_2', 'T28_wa2b_rhh2_3', 'T27_wa3_dhh1_1', 'T25_wa3_dhh1_2', 'T23_wa3_rhh1_2', 'T21_wa3_rhh1_3', 'T37_wa3_dhh3_1', 'T35_wa3_dhh3_2', 'T33_wa3_dhh2_1', 'T29_wa3_dhh2_2', 'T43_wa3_dhh5_1', 'T41_wa3_dhh5_2', 'T45_wa3_dhh6_1', 'T49_wa3_rhh2_2', 'T47_wa3_rhh2_3', 'T26_wa2b_rhh1_1', 'T24_wa2b_rhh1_2', 'T22_wa2b_rhh1_3', 'T18_wa2b_dhh1_1']

🔧 PUMP ANALYSIS RESULTS
                     Pump Power_T56_wa2b_efh2  Pump Power_T54_wa2b_efh1  \
2024-01-01 00:00:00                  0.000538                 29.349482   
2024-01-01 00:15:00                  0.